In [1]:
import pandas as pd
df=pd.read_csv('new_data.csv')

df=df[(df.type_char!='file_folder')&(df.type_char!='handwritten')]

df.type_char.replace(['facture', 'justif_domicile','paye',
       'rrc.cvc', 'form', 'scientific_publication','letter',
       'advertisement', 'resume', 'specification', 'memo',
       'invoice', 'budget', 'news_article', 'presentation',
       'scientific_report', 'email', 'questionnaire'],['texte']*18,inplace=True)

df['type']=df.type_char.replace({'id_pieces':0,'passeport':1,'carte postale':2,'texte':3})
    
df.head()


,Unnamed: 0,filename,type,text_image,type_char
0,0,img_0000000.jpg,3,FACTURE\n\nLOGO\n\nJoanna Binet\n48 Coubertin\...,texte
1,1,img_0000001.jpg,3,Joanna Binet\n48 Coubertin\n31400 Paris\n\nFAC...,texte
2,2,img_0000002.jpg,3,FACTURE\n\nMon entreprise : Nom de la société\...,texte
3,3,img_0000003.jpg,3,Joanna Binet\n48 Coubertin\n31400 Paris\n\nFAC...,texte
4,4,img_0000004.jpg,3,Payer en ligne >\nFACTURE No\n\nSFIDELI\n\nF/0...,texte


In [4]:
# importer les image sous forme d'un tableau
import cv2 as cv
import numpy as np
list_img=[]
for img_n  in df.filename :
    img=cv.imread('final/'+img_n)
    #gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    img= cv.resize(img, (64, 64))
    list_img.append(img)
X=np.array(list_img)
X=X/255

In [5]:
#df['type_nbr']=df.type
#df.type_nbr.replace(['facture', 'id_pieces', 'justif_domicile', 'passeport', 'paye',
       #'carte postale', 'rrc.cvc', 'form', 'scientific_publication',
       #'advertisement', 'letter', 'resume', 'specification',
        #'memo', 'invoice', 'budget', 'news_article',
       #'presentation', 'scientific_report', 'email',
       #'questionnaire'],[i for i in range(21)],inplace=True)
Y=np.array(df.type)

In [6]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
X_train, X_test, Y_train ,Y_test =train_test_split(X,Y,test_size=0.2)
y_train=to_categorical(Y_train)
y_test=to_categorical(Y_test)

In [7]:
X_train =X_train.reshape((-1,64,64,3))
X_test =X_test.reshape((-1,64,64,3))

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen=ImageDataGenerator(zoom_range=0.4,horizontal_flip=True,shear_range=0.1)
test_datagen=ImageDataGenerator()

In [9]:
train_dataset=train_datagen.flow(X_train,y_train,batch_size=300)
test_dataset=test_datagen.flow(X_test,y_test,batch_size=128)

In [10]:
%matplotlib inline
import numpy as np

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Activation, Input
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D


import matplotlib.pyplot as plt

In [11]:
inputs = Input(shape = (64,64,3), name = "Input")
conv_1=Conv2D(filters = 50,kernel_size = (2, 2), padding = 'valid',activation = 'relu')
max_Pool_1=MaxPooling2D(pool_size = (2, 2))
conv_2=Conv2D(filters = 26,kernel_size = (2, 2), padding = 'valid',activation = 'relu')
max_Pool_2=MaxPooling2D(pool_size = (2, 2))
dropout=Dropout(rate = 0.2)
aplat=Flatten()
dense_1=Dense(units = 36, activation = "relu")
dense_2=Dense(units = 4, activation = "softmax")
x=conv_1(inputs)
x=max_Pool_1(x)
x=conv_2(x)
x=max_Pool_2(x)
x=dropout(x)
x=aplat(x)
x=dense_1(x)
outputs=dense_2(x)
model= Model(inputs = inputs, outputs = outputs)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 64, 64, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 63, 63, 50)        650       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 50)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 30, 30, 26)        5226      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 15, 15, 26)       0         
 2D)                                                             
                                                                 
 dropout (Dropout)           (None, 15, 15, 26)        0     

In [12]:
model.compile(loss="categorical_crossentropy",optimizer = "adam", metrics = ["accuracy"])
training_history_lenet=model.fit_generator(train_dataset,
                            epochs = 100,
                            steps_per_epoch = len(X_train)*0.8 // 300)
                             
#training_history_lenet=model.fit(train_dataset,y_train,epochs = 200, batch_size = 200, validation_split = 0.2)

C:\Users\sadoun\AppData\Local\Temp\ipykernel_16320\3889636594.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  training_history_lenet=model.fit_generator(train_dataset,


Epoch 1/100
3/3 [==============================] - 3s 331ms/step - loss: 1.1715 - accuracy: 0.5528
Epoch 2/100
3/3 [==============================] - 2s 378ms/step - loss: 0.9993 - accuracy: 0.6740
Epoch 3/100
3/3 [==============================] - 2s 520ms/step - loss: 0.9186 - accuracy: 0.6898
Epoch 4/100
3/3 [==============================] - 2s 489ms/step - loss: 0.8691 - accuracy: 0.6961
Epoch 5/100
3/3 [==============================] - 2s 755ms/step - loss: 0.8394 - accuracy: 0.7118
Epoch 6/100
3/3 [==============================] - 2s 613ms/step - loss: 0.8596 - accuracy: 0.6944
Epoch 7/100
3/3 [==============================] - 2s 482ms/step - loss: 0.8512 - accuracy: 0.6929
Epoch 8/100
3/3 [==============================] - 2s 345ms/step - loss: 0.8914 - accuracy: 0.6756
Epoch 9/100
3/3 [==============================] - 2s 757ms/step - loss: 0.7824 - accuracy: 0.7228
Epoch 10/100
3/3 [==============================] - 2s 341ms/step - loss: 0.8422 - accuracy: 0.6850
Epoch 11/

3/3 [==============================] - 3s 724ms/step - loss: 0.5228 - accuracy: 0.7967
Epoch 84/100
3/3 [==============================] - 2s 638ms/step - loss: 0.5010 - accuracy: 0.8111
Epoch 85/100
3/3 [==============================] - 2s 673ms/step - loss: 0.5213 - accuracy: 0.8167
Epoch 86/100
3/3 [==============================] - 2s 650ms/step - loss: 0.4846 - accuracy: 0.8189
Epoch 87/100
3/3 [==============================] - 2s 539ms/step - loss: 0.4676 - accuracy: 0.8157
Epoch 88/100
3/3 [==============================] - 2s 721ms/step - loss: 0.5175 - accuracy: 0.8144
Epoch 89/100
3/3 [==============================] - 2s 645ms/step - loss: 0.4716 - accuracy: 0.8233
Epoch 90/100
3/3 [==============================] - 2s 480ms/step - loss: 0.5008 - accuracy: 0.8220
Epoch 91/100
3/3 [==============================] - 2s 328ms/step - loss: 0.4984 - accuracy: 0.8126
Epoch 92/100
3/3 [==============================] - 2s 498ms/step - loss: 0.5075 - accuracy: 0.8126
Epoch 93/100


In [13]:
from sklearn.metrics import classification_report

test_pred_lenet=model.predict(X_test)
test_pred_lenet_class=test_pred_lenet.argmax(axis=1)
y_test_class=y_test.argmax(axis=1)
print(classification_report(y_test_class, test_pred_lenet_class))

10/10 [==============================] - 0s 22ms/step
              precision    recall  f1-score   support

           0       0.64      0.29      0.40        24
           1       0.50      0.38      0.43        13
           2       0.62      0.33      0.43        39
           3       0.85      0.97      0.91       233

    accuracy                           0.82       309
   macro avg       0.65      0.50      0.54       309
weighted avg       0.79      0.82      0.79       309

